In [ ]:
import numpy as np
import pandas as pd
import mikeio
from mikeio import ItemInfo, EUMType, EUMUnit

# Write a dfs0

A mikeio.Dataset contains the information needed to write a dfs file. A Dataset consists of one or more mikeio.DataArrays each corresponding to an "item" in a dfs file.

In [ ]:
nt = 10
time = pd.date_range("2000-1-1", periods=nt, freq="H")

d1 = np.zeros(nt)
item = ItemInfo("Zeros", EUMType.Water_Level)
da1 = mikeio.DataArray(d1, time=time, item=item)

d2 = np.ones(nt)
item = ItemInfo("Ones", EUMType.Discharge, EUMUnit.meter_pow_3_per_sec)
da2 = mikeio.DataArray(d2, time=time, item=item)

ds = mikeio.Dataset([da1, da2])
ds

In [ ]:
ds.is_equidistant

In [ ]:
ds.to_dfs("test.dfs0", title="Zeros and ones")

# Read a timeseries

A dfs file is easily read with mikeio.read which returns a Dataset.

In [ ]:
ds = mikeio.read("test.dfs0")
ds

## From comma separated file

In [ ]:
df = pd.read_csv(
    "../tests/testdata/co2-mm-mlo.csv",
    parse_dates=True,
    index_col="Date",
    na_values=-99.99,
)
df.head()

Remove missing values

In [ ]:
df = df.dropna()
df = df[["Average", "Trend"]]
df.plot()

A dataframe with a datetimeindex can be used to create a dfs0 with a non-equidistant time axis by first converting it to a mikeio.Dataset.

In [ ]:
ds = mikeio.from_pandas(df)
ds

And then write to a dfs0 file:

In [ ]:
ds.to_dfs("mauna_loa_co2.dfs0")

To get a equidistant time axis first interpolate to regularly spaced values, in this case daily.

*The code for this can be written in many ways, below is an example, where we avoid temporary variables.*

In [ ]:
(
    df.resample("D")  # resample to daily
    .interpolate()  # interpolate linearly
    .pipe(mikeio.from_pandas)  # convert to mikeio.Dataset
    .to_dfs("mauna_loa_co2_daily.dfs0")  # save to dfs0
)

# Read a timeseries

In [ ]:
res = mikeio.read("test.dfs0")
res

In [ ]:
res.time

In [ ]:
res.to_numpy()

## Or as a Pandas dataframe

A mikeio.Dataset ds is converted to a pandas dataframe with ds.to_dataframe()

In [ ]:
dfs0file = "../tests/testdata/da_diagnostic.dfs0"
df = mikeio.read(dfs0file).to_dataframe()
df.head()

In [ ]:
dfs0file = "../tests/testdata/random.dfs0"
df = mikeio.read(dfs0file).to_dataframe()
df.head()

## Create a timeseries with non-equidistant data

In [ ]:
d1 = np.random.uniform(low=0.0, high=5.0, size=5)
time = pd.DatetimeIndex(
    ["2000-1-1", "2000-1-8", "2000-1-10", "2000-2-22", "2000-11-29"]
)
da = mikeio.DataArray(d1, time=time, item=ItemInfo("Random"))
da

In [ ]:
da.is_equidistant

In [ ]:
da.to_dfs("neq.dfs0", title="Non equidistant")

## Create a timeseries with accumulated timestep

## Find correct eum units

In [ ]:
EUMType.search("prec")

In [ ]:
EUMType.Precipitation_Rate.units

In [ ]:
from mikecore.DfsFile import DataValueType

n = 1000
time = pd.date_range("2017-01-01 00:00", freq="H", periods=n)

# use default name and unit based on type
item = ItemInfo(EUMType.Water_Level, data_value_type=DataValueType.Instantaneous)
da1 = mikeio.DataArray(data=np.random.random([n]), time=time, item=item)

# use a custom name
item = ItemInfo(
    "Nedbør", EUMType.Precipitation_Rate, data_value_type=DataValueType.Accumulated
)
da2 = mikeio.DataArray(data=np.random.random([n]), time=time, item=item)

ds = mikeio.Dataset([da1, da2])
ds.to_dfs("accumulated.dfs0")

In [ ]:
ds = mikeio.read("accumulated.dfs0")
ds

# Modify an existing timeseries

In [ ]:
ds = mikeio.read("test.dfs0")
ds

In [ ]:
ds["Ones"]

Modify the data in some way...

In [ ]:
ds["Ones"] = ds["Ones"] * np.pi
ds["Ones"].values

In [ ]:
ds.to_dfs("modified.dfs0")

In [ ]:
res = mikeio.read("modified.dfs0")
res["Ones"]

The second item is not modified.

In [ ]:
res["Zeros"]

## Convert units

Read a file with waterlevel i meters.

In [ ]:
filename = "../tests/testdata/waterlevel_viken.dfs0"
# filename = r"C:\Program Files (x86)\DHI\MIKE Zero\2021\Examples\MIKE_21\FlowModel_FM\HD\Oresund\Data\1993\Boundary_Conditions\waterlevel_viken.dfs0"
ds = mikeio.read(filename)
ds

In [ ]:
ds.plot()

The aim is to convert this timeseries to feet (1m = 3.3 ft)

In [ ]:
ds[0] = ds[0] * 3.3

Which units are acceptable?

In [ ]:
ds.items[0].type.units

In [ ]:
ds[0].item = ItemInfo("Viken", ds[0].item.type, EUMUnit.feet)

In [ ]:
ds.to_dfs("wl_feet.dfs0")

![WL](https://github.com/DHI/mikeio/raw/main/images/wl_feet.png)

## Extrapolation

In [ ]:
# filename = r"C:\Program Files (x86)\DHI\MIKE Zero\2021\Examples\MIKE_21\FlowModel_FM\HD\Oresund\Data\1993\Boundary_Conditions\waterlevel_viken.dfs0"
filename = "../tests/testdata/waterlevel_viken.dfs0"
ds = mikeio.read(filename)
df = ds.to_dataframe()
df.plot()

In [ ]:
rng = pd.date_range("1993-12-1", "1994-1-1", freq="30t")
ix = pd.DatetimeIndex(rng)
dfr = df.reindex(ix)
dfr.plot()

Replace NaN with constant extrapolation (forward fill + back fill).

In [ ]:
dfr = dfr.ffill().bfill()
dfr.plot()

In [ ]:
(
    mikeio.from_pandas(dfr, items=ds.items).to_dfs(
        "Viken_extrapolated.dfs0", title="Caution extrapolated data!"
    )
)

## Clean up

In [ ]:
import os

os.remove("test.dfs0")
os.remove("modified.dfs0")
os.remove("neq.dfs0")
os.remove("accumulated.dfs0")
os.remove("wl_feet.dfs0")
os.remove("mauna_loa_co2_daily.dfs0")
os.remove("mauna_loa_co2.dfs0")
os.remove("Viken_extrapolated.dfs0")